In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> all
Command 'all' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_data...
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Un


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

## Gender Identification
Gender identification, in the context of natural language processing, is the task of determining the gender of a person based on their names. This task can be achieved through supervised classification using nltk, where the model is trained on a labeled dataset containing name samples along with their corresponding gender labels (e.g., male or female).

##Step1 : Feature Extractor

The first step in creating classifier, is to decide what features of inputs are relevant and how to encoding those features. For instance i am start by looking at lastletter of given name.
The returned dictionary known as feature sets which gives basic information about inputs.


In [ ]:
def gender_features(name):
  return {'last_letter':name[-1]}

gender_features('madhu')

{'last_letter': 'u'}

##Step2 : Data Preparation
Now we have to prepare a list of example names with its corresponding gender labels and shuffle the names using random library.


In [ ]:
from nltk.corpus import names
labeled_names = [(name,'male') for name in names.words('male.txt')] + [(name,'female') for name in names.words('female.txt')]
import random
random.shuffle(labeled_names)

##Step 3 : Data Splitting
Now use the feature extractor to process the labeled names and divide the resulting feature sets to testing data and test data.



In [ ]:
feature_sets = [(gender_features(n),gender) for (n,gender) in labeled_names]
train_set,test_set = feature_sets[500:],feature_sets[:500]

##Step 4 : Model Training
Here i am selecting Naive Bayes algorithm to build classifier model by training on training set.


In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

##Step 5 : Model Prediction
Now see how the model predict the gender labels for given name.


In [ ]:
classifier.classify(gender_features('Sania'))

'female'

##Step 5 : Model Evaluation
Now evaluate the model performance by using unseen, new test data.

In [ ]:
nltk.classify.accuracy(classifier,test_set)

0.744

In [ ]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'k'              male : female =     44.2 : 1.0
             last_letter = 'a'            female : male   =     36.8 : 1.0
             last_letter = 'f'              male : female =     27.6 : 1.0
             last_letter = 'p'              male : female =     11.1 : 1.0
             last_letter = 'd'              male : female =     10.6 : 1.0


This listing shows that the names in the training set that end in "k" are male 44 times more often than they are female, but names that end in "a" are female 36 times more often than they are male. These ratios are known as likelihood ratios, and can be useful for comparing different feature-outcome relationships.

Once an initial set of features has been chosen, a very productive method for refining the feature set is error analysis. First, we select a development set, containing the corpus data for creating the model. This development set is then subdivided into the training set and the dev-test set.




In [ ]:
train_set = [(gender_features(n), gender) for (n, gender) in labeled_names[1500:]]
devtest_set = [(gender_features(n), gender) for (n, gender) in labeled_names[500:1500]]
test_set = [(gender_features(n), gender) for (n, gender) in labeled_names[:500]]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)


Using the dev-test set, we can generate a list of the errors that the classifier makes when predicting name genders:


In [ ]:
errors = []
for (name, tag) in labeled_names[500:1500]:
  guess = classifier.classify(gender_features(name))
  if guess != tag:
      errors.append( (tag, guess, name) )

In [ ]:
for (tag, guess, name) in sorted(errors):
  print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Aigneis                       
correct=female   guess=male     name=Allsun                        
correct=female   guess=male     name=Ambur                         
correct=female   guess=male     name=Annabal                       
correct=female   guess=male     name=Bess                          
correct=female   guess=male     name=Brandais                      
correct=female   guess=male     name=Brear                         
correct=female   guess=male     name=Brook                         
correct=female   guess=male     name=Cal                           
correct=female   guess=male     name=Camel                         
correct=female   guess=male     name=Caro                          
correct=female   guess=male     name=Charin                        
correct=female   guess=male     name=Charmian                      
correct=female   guess=male     name=Chicky                        
correct=female   guess=male     name=Christean  

Looking through this list of errors makes it clear that some suffixes that are more than one letter can be indicative of name genders. For example, names ending in yn appear to be predominantly female, despite the fact that names ending in n tend to be male; and names ending in ch are usually male, even though names that end in h tend to be female. We therefore adjust our feature extractor to include features for two-letter suffixes

In [ ]:
def gender_features(word):
     return {'suffix1': word[-1:],
             'suffix2': word[-2:]}

In [ ]:
train_set = [(gender_features(n),gender) for (n,gender) in labeled_names[500:]]
test_set = [(gender_features(n),gender) for (n,gender) in labeled_names[:500]]


In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
nltk.classify.accuracy(classifier,test_set)

0.778